In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
vars_procedimientos = {f"PROCEDIMIENTO{i}": str for i in range(1, 31)}

In [4]:
df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_7140\2020992259.py:1: DtypeWarning: Columns (24,25,26,27,28,29,30,31,32,33,37,41,43,47,51,75,76,77,78,79,80,81,82,83,84,85,86,87,123) have mixed types. Specify dtype option on import or set low_memory=False.
  df_procesada = pd.read_csv("../data/processed/df_procesada.csv", sep=";", dtype=vars_procedimientos)


In [41]:
COLUMNAS_SERVICIO_TRASLADO = {
    "SERVICIOINGRESO": 0,
    "SERVICIOTRASLADO1": 1,
    "SERVICIOTRASLADO2": 2,
    "SERVICIOTRASLADO3": 3,
    "SERVICIOTRASLADO4": 4,
    "SERVICIOTRASLADO5": 5,
    "SERVICIOTRASLADO6": 6,
    "SERVICIOTRASLADO7": 7,
    "SERVICIOTRASLADO8": 8,
    "SERVICIOTRASLADO9": 9,
    "SERVICIOALTA": 10,
}

COLUMNAS_FECHA_TRASLADO = {
    "FECHA_INGRESO": 0,
    "FECHATRASLADO1": 1,
    "FECHATRASLADO2": 2,
    "FECHATRASLADO3": 3,
    "FECHATRASLADO4": 4,
    "FECHATRASLADO5": 5,
    "FECHATRASLADO6": 6,
    "FECHATRASLADO7": 7,
    "FECHATRASLADO8": 8,
    "FECHATRASLADO9": 9,
    "FECHAALTA": 10,
}


# Filtra la base nacional para dejar el Torax
df_torax = df_procesada.query("COD_HOSPITAL == 112103")
# Permite identificar cada egreso
df_torax = df_torax.reset_index().reset_index(names="id_egreso")

In [48]:
# Convierte los servicios y las fechas en formato largo
servicios_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_SERVICIO_TRASLADO,
    var_name="tipo_servicio",
    value_name="servicio",
)

fechas_egreso_long = df_torax.melt(
    id_vars="id_egreso",
    value_vars=COLUMNAS_FECHA_TRASLADO,
    var_name="tipo_fecha",
    value_name="fecha",
)

In [90]:
# Une los servicios y fechas respectivas para que esten en formato contiguo
servicio_y_fechas_egreso_contiguas = pd.merge(
    left=fechas_egreso_long,
    right=servicios_egreso_long,
    how="inner",
    left_index=True,
    right_index=True,
)

# Indica el tipo de ordenamiento del tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
servicio_y_fechas_egreso_contiguas["tipo_servicio"] = pd.Categorical(
    servicio_y_fechas_egreso_contiguas["tipo_servicio"],
    categories=COLUMNAS_SERVICIO_TRASLADO.keys(),
    ordered=True,
)

# Ordena cada egreso segun su tipo de servicio ("SERVICIONGR", "SERVICIO1", "SERVICIOALTA")
servicio_y_fechas_egreso_contiguas = servicio_y_fechas_egreso_contiguas.sort_values(
    ["id_egreso_x", "tipo_servicio"]
)

# Elimina filas sin ninguna fecha ingresada
servicio_y_fechas_egreso_contiguas = servicio_y_fechas_egreso_contiguas.dropna()

# Imputa fechas con formato de fechas
servicio_y_fechas_egreso_contiguas["fecha"] = pd.to_datetime(
    servicio_y_fechas_egreso_contiguas["fecha"]
)

In [91]:
servicio_y_fechas_egreso_contiguas["tiempo_de_estancia"] = (
    servicio_y_fechas_egreso_contiguas.groupby("id_egreso_x")["fecha"].diff().shift(-1)
)

In [96]:
servicio_y_fechas_egreso_contiguas.groupby("servicio")["tiempo_de_estancia"].describe()

,count,mean,std,min,25%,50%,75%,max
servicio,,,,,,,,
AREA MEDICA ADULTO CUIDADOS BASICOS,1,1 days 00:00:00,NaT,1 days 00:00:00,1 days 00:00:00,1 days 00:00:00,1 days 00:00:00,1 days 00:00:00
AREA MEDICA ADULTO CUIDADOS MEDIOS,1128,5 days 09:35:44.680851063,7 days 02:05:30.125415732,0 days 00:00:00,1 days 00:00:00,3 days 00:00:00,7 days 00:00:00,116 days 00:00:00
AREA MEDICO-QUIRURGICO CUIDADOS MEDIOS,1707,4 days 07:27:06.010544815,10 days 14:54:12.060519951,0 days 00:00:00,1 days 00:00:00,2 days 00:00:00,5 days 00:00:00,393 days 00:00:00
AREA MÉDICA,3751,4 days 00:28:24.505465209,13 days 03:08:10.050473058,-344 days +00:00:00,0 days 00:00:00,1 days 00:00:00,4 days 00:00:00,374 days 00:00:00
AREA QUIRÚRGICA,3390,5 days 04:17:50.442477876,15 days 23:07:20.131616126,0 days 00:00:00,1 days 00:00:00,2 days 00:00:00,6 days 00:00:00,394 days 00:00:00
CIRUGÍA CARDIOVASCULAR,2302,4 days 07:51:39.565595134,11 days 17:23:11.354616749,0 days 00:00:00,1 days 00:00:00,2 days 00:00:00,5 days 00:00:00,368 days 00:00:00
UNIDAD DE CUIDADOS INTENSIVOS (UCI) (INDIFERENCIADO),386,4 days 03:02:47.875647668,33 days 19:30:33.167311763,-359 days +00:00:00,2 days 00:00:00,3 days 00:00:00,7 days 00:00:00,77 days 00:00:00
UNIDAD DE CUIDADOS INTENSIVOS ADULTO,2417,3 days 19:42:01.638394704,30 days 04:39:26.491819526,-363 days +00:00:00,2 days 00:00:00,3 days 00:00:00,5 days 00:00:00,411 days 00:00:00
UNIDAD DE CUIDADOS INTENSIVOS CARDIOLOGÍA,455,14 days 11:33:05.934065934,15 days 10:01:37.342349028,0 days 00:00:00,4 days 00:00:00,10 days 00:00:00,20 days 00:00:00,126 days 00:00:00


In [112]:
egresos_con_tiempos_erroneos = list(
    servicio_y_fechas_egreso_contiguas[
        servicio_y_fechas_egreso_contiguas["tiempo_de_estancia"] < np.timedelta64(0)
    ]["id_egreso_x"]
)

In [104]:
servicio_y_fechas_egreso_contiguas.query("id_egreso_x == 1380")

,id_egreso_x,tipo_fecha,fecha,id_egreso_y,tipo_servicio,servicio,tiempo_de_estancia
1380,1380,FECHA_INGRESO,2018-10-04,1380,SERVICIOINGRESO,AREA MÉDICA,7 days
14846,1380,FECHATRASLADO1,2018-10-11,1380,SERVICIOTRASLADO1,UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO,-283 days
28312,1380,FECHATRASLADO2,2018-01-01,1380,SERVICIOTRASLADO2,UNIDAD DE CUIDADOS INTENSIVOS ADULTO,368 days
41778,1380,FECHATRASLADO3,2019-01-04,1380,SERVICIOTRASLADO3,UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR,12 days
55244,1380,FECHATRASLADO4,2019-01-16,1380,SERVICIOTRASLADO4,UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO,21 days
68710,1380,FECHATRASLADO5,2019-02-06,1380,SERVICIOTRASLADO5,AREA MÉDICA,18 days
136040,1380,FECHAALTA,2019-02-24,1380,SERVICIOALTA,AREA MÉDICA,NaT


In [109]:
df_torax.query("id_egreso == 1380").iloc[:, 15:30]

,TIPO_ACTIVIDAD,FECHA_INGRESO,SERVICIOINGRESO,FECHATRASLADO1,SERVICIOTRASLADO1,FECHATRASLADO2,SERVICIOTRASLADO2,FECHATRASLADO3,SERVICIOTRASLADO3,FECHATRASLADO4,SERVICIOTRASLADO4,FECHATRASLADO5,SERVICIOTRASLADO5,FECHATRASLADO6,SERVICIOTRASLADO6
1380,HOSPITALIZACIÓN,2018-10-04,AREA MÉDICA,2018-10-11,UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO,2018-01-01,UNIDAD DE CUIDADOS INTENSIVOS ADULTO,2019-01-04,UNIDAD DE TRATAMIENTO INTERMEDIO CARDIOVASCULAR,2019-01-16,UNIDAD DE TRATAMIENTO INTERMEDIO CIRUGÍA ADULTO,2019-02-06,AREA MÉDICA,NaN,NaN
